In [18]:
from src.task.optimus import OptimusTask
from faiss import IndexFlatL2, write_index
from datasets import load_dataset, load_from_disk

In [2]:
%%capture
task = OptimusTask.load_from_checkpoint("checkpoint/optimus-aihub_conv-396k.ckpt", map_location='cuda:0')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some wei

In [88]:
dataset = load_dataset("json", data_files="data/dialog/aihub_daily_conv_train.jsonl", split="train")
dataset = dataset.filter(lambda x: len(x['text']) > 0)
# dataset = dataset.map(lambda x: {"latent": task.encode(x["text"][0]).numpy()})
# dataset.save_to_disk("latent512.hfds")

Using custom data configuration default-727aceca261e332e
Reusing dataset json (/home/ubuntu/.cache/huggingface/datasets/json/default-727aceca261e332e/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/json/default-727aceca261e332e/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253/cache-e8ee90a7b5ad4925.arrow


In [ ]:
dataset = load_from_disk("./latent512.hfds/")

In [89]:
faiss = IndexFlatL2(task.config.model.latent_dim)

In [ ]:
from tqdm import tqdm
import numpy as np

for i in tqdm(range(len(dataset))):
    text = dataset[i]['text']
    latent = task.encode(text).numpy()
    faiss.add_w(latent)
    # if i == 100:
    #     break

In [ ]:
write_index(faiss, "ckpt396-512d.faiss")

In [131]:
context = "[사용자1]: 이번주말에는 영화나 보러갈까?\n"
z = task.encode(context)

In [132]:
k = 2
dist, indices = faiss.search(z.numpy(), k)
dist, indices

(array([[60.057644, 62.20302 ]], dtype=float32), array([[14368,  4686]]))

In [133]:
import torch

Zr = torch.zeros_like(z, device=task.device)
Zr += z
t = k + 1

for idx in indices[0]:
    idx = int(idx)
    
    if idx in [101]:
        t -= 1
        continue

    print(idx, dataset[idx]["text"].replace('\n', ''))

    
    Zr += task.encode(dataset[idx]['text'])[0]
    

Zr = Zr / t
# Zr

14368 [사용자1]: 이번에 대구 내려가는 기차표 구하기 너무 힘들었어[사용자2]: 음...미리 예매 안 했어?워낙 이동하는 인구가 많으니깐[사용자1]: 그래도 겨우 겨우 구하긴 했는데! 못 구했으면 버스 타고 내려갈 뻔...[사용자2]: 헐... 서울에서 대구까지 버스면 너무 힘들지기차표 구했다니깐 다행이야![사용자1]: 비행기표도 알아봤었는데 대구까지 가는 건 기차표 2배 정도더라고...[사용자2]: 헐... 그렇게나 비싸? 비행기 값이?기차표 구하길 잘했네! 다행이다![사용자1]: 응 ㅜ 대구까지는 기차가 싸더라구... 그래도 대기표 구했으니 다행이지![사용자2]: 오오~다행이야! 대기표라도 있었다니 예매 해 놓고 취소한 사람들인가?[사용자1]: 웅웅 그런가 봐! 한 자리니까 그런 표 자주 나오더라![사용자2]: 아아~ 취소할 거면 왜 예매를 했대? ㅋㅋ[사용자1]: 아휴... 다들 사정이 있겠지 뭐... ㅋㅋ 나야 취소표 나왔으니 좋고![사용자2]: ㅋㅋ 어쨌든 너야 잘 왔으니깐 된 거지 ㅋㅋ[사용자1]: 응응! 담에 명절 때 말고 비행기표 저렴하면 비행기 타고도 한번 와봐야겠다![사용자2]: ㅋㅋ 기차표보단 무조건 비쌀 거 같은데?기차 타고 오는 게 제일 좋을 거 같아 ㅋㅋ[사용자1]: 근데 동대구역이 집이랑 가까워 대구공항이 집이랑 가까워?[사용자2]: 동대구역이랑 가깝지 공항은 버스로 와야 되고 역은 지하철이라도 있으니깐
4686 [사용자1]: 엄마 **가 절뚝 거리는 데요?[사용자2]: 아까 나올 때까지만 해도 괜찮았는데.[사용자1]: 병원 데려가야 할 것 같아요[사용자2]: 갑자기 무슨 일이라니...[사용자1]: 아까는 괜찮았어요?[사용자2]: 그렇단다 잘 있는 거 보고 나왔는데.[사용자1]: 내일 병원 데려가야겠어요.[사용자2]: 그래야겠다 별일 아니겠지?[사용자1]: 오늘 하루 좀 지켜봐야겠어요.[사용자2]: 우리 ** 큰 일 아니겠지 걱정된다.[사용자1]: 큰 일 아니길 바래야죠.[사용자2]: 엄마 금방 들어 갈게 ** 잘 보고 있어.

In [134]:
task.generate(Zr, "<|endoftext|> " + context + "[사용자2]: ", num_beams=5, max_length=128)
# task.generate(Zr, prompts=None, num_beams=4, max_length=128, repetition_penalty=2.0)

['<|endoftext|> [사용자1]: 이번주말에는 영화나 보러갈까?\n[사용자2]: 헉... 기차표 보니까 너무 귀찮아.\n[사용자1]: 너희 집 근처라도 갔다 왔으면 좋겠어?\n[사용자2]: 아무래도 환승해야 할 거 같아.\n[사용자1]: 그런가? 우리 동네는 금액은 얼마 정도 나와?\n[사용자2]: 글쎄. 아마 천차만별이고 있을 거야.\n[사용자1]: 알겠어요일에 보러 갈까?\n[사용자2]: 그래야지. 이번 주말에 보러 갈']

In [ ]:
a, b = task.encode(["안녕하세요", "반갑습니다"])
a - b

In [83]:
a = np.array(dataset[5169]["latent"][0])
b = np.array(dataset[5924]["latent"][0])
c = np.array(dataset[9005]["latent"][0])

# np.dot(a, b), np.dot(a, c), np.dot(b, c)
a - b

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.